# Name: Falak Jain

# Problem Set 10

### Learning Objective:

- Create Python code to automate a given task.
- Formulate linear optimization models to inform a business decision.

### Overview:

This problem set assesses your ability to turn an abstract formulation into reusable optimization software, as discussed in Week 12.

### Grading

There are three possible scores you can get from submitting this assignment on time (submitting a blank file or one without any apparent effort does not count). Note that the rubric is designed to incentivize you to go for 100% mastery of the material, as the little details matter a lot in business analytics. 

| Grade | Description |
|--|--|
| 5 out of 5 | Perfect submission with no significant errors. | 
| 4 out of 5 | Near perfect submission with one or more significant errors. |
| 2 out of 5 | Apparent effort but far from perfect. |

## Q1. Assigning of Final Grades

This question asks you to create software that can help a professor assign final grades in such a way so that the average GPA rounds to 3.5, while obtaining an assignment in which there are gaps in scores between consecutive grade levels, and no particular grade is assigned to disproportionally many students.

**Data:** 

- $I$: the set of students.
- $n$: the number of students.
- $J=\{0,1,\cdots\}$: numerical indices used to denote the various grade levels.
- $s_i$: the overall score of student $i\in I$ (between 0 and 100). 
- $g_j$: the GPA corresponding to grade level $j \in J$.

**Decision Variables:**

- $x_{ij}$: whether to assign student $i$ to grade level $j$. (Binary)
- $t_j$: the number of students assigned to grade level $j$. (Continuous)
- $L_j$: the score cutoff for grade level $j$. (Continuous)
- $U_j$: the maximum score in grade level $j$. (Continuous)

$$\begin{aligned}
\text{Min} && \sum_{j \in J}(U_j-L_j) + 0.1 \sum_{j \in J} t_j \times t_j \\
\text{s.t.} \\
\text{(Average GPA)} && 3.495n \le \sum_{i \in I,j \in J} x_{ij}g_{j} & \le 3.505n \\
\text{(Assignment)} && \sum_{j \in J} x_{ij} & = 1 && \text{for each $i \in I$.}\\
\text{(Max score)} && s_i x_{ij} & \le U_j && \text{for each $i \in I$, $j \in J$.}\\
\text{(Min score)} && 100(1-x_{ij}) + s_i x_{ij} & \ge L_j && \text{for each $i \in I$, $j \in J$.}\\
\text{(Correct totals)} && \sum_{i \in I} x_{ij} & = t_j && \text{for each $j \in J$.}\\
\text{(Bounds)} && L_j & \le U_j && \text{for each $j \in J$.} \\
\text{(Ordering)} && U_j & \le L_{j-1} && \text{for each $j \in J$ with $j \ge 1$.}\\
&& t_j, L_j, U_j & \ge 0 && \text{for each $j \in J$.}
\end{aligned}$$

The input data is contained in an Excel file named `PS10-grade-input.xlsx` with two sheets. The first sheet, named "Scores", contains the score of each student. The first five entries look like:

![Sample Scores Sheet](PS10-grade1.png)

The second sheet, named "Levels", is as follows

![Sample Levels Sheet](PS10-grade2.png)

The output data should be an Excel file named `PS10-grade-output.xlsx` that contains the cutoff ($L_j$) for each grade level $j$. It should look like

![Sample Output](PS10-grade3.png)



In [75]:
# Write your code here
scores = pd.read_excel('PS10-grade-input.xlsx', sheet_name = "Scores",index_col = 0)
levels = pd.read_excel('PS10-grade-input.xlsx', sheet_name = "Levels", index_col = 0)
I = scores.index
J = levels.index
n = len(scores)
mod = Model()
x = mod.addVars(I,J,vtype = GRB.BINARY,name = 'x')
t = mod.addVars(J,vtype = GRB.CONTINUOUS,name = 't')
L = mod.addVars(J,vtype = GRB.CONTINUOUS,name = 'L')
U = mod.addVars(J,vtype = GRB.CONTINUOUS,name = 'U')
mod.setObjective(sum((U[j]-L[j]) for j in J)+0.1*sum((t[j]*t[j])for j in J), sense = GRB.MINIMIZE)    # shift requirement
# average GPA
mod.addConstr(sum(x[i,j]*levels.loc[j,'g_j'] for i in I for j in J) <= 3.505*n,name='Average GPA 1')
mod.addConstr(sum(x[i,j]*levels.loc[j,'g_j'] for i in I for j in J) >= 3.495*n,name='Average GPA 2')
# assignment
for i in I:
    mod.addConstr(sum(x[i,j] for j in J) == 1,name='Assignment')
# max score
for i in I:
    for j in J:
        mod.addConstr(scores.loc[i,'s_i']*x[i,j]<=U[j],name='max score')
# min score
for i in I:
    for j in J:
        mod.addConstr(100*(1-x[i,j])+scores.loc[i,'s_i']*x[i,j]>=L[j],name='max score')
# correct totals
for j in J:
    mod.addConstr(sum(x[i,j] for i in I) == t[j],name='Correct Totals')
# bounds
for j in J:
    mod.addConstr(L[j]<=U[j],name='bounds')
# ordering
for j in range(1,len(J)):
    mod.addConstr(U[j]<=L[j-1],name='ordering')
mod.setParam("OutputFlag", False)
mod.setParam('MIPGap',1e-6)
mod.optimize()
mod.objVal
df = pd.DataFrame(index = levels.Letter,columns = ['Cutoff'])
for j in J:
    df.loc[levels.loc[j,'Letter'],'Cutoff'] = L[j].x
df.Cutoff = df.Cutoff.astype(int)
df.reset_index(inplace = True)
writer = pd.ExcelWriter('PS10-grade-output.xlsx')
df.to_excel(writer, sheet_name = "Sheet1", index = False)
writer.save()

## Q2. Team Assignment

The following MIP can used to assign students into project teams to balance the overall characteristics of each team.

**Data:**

- $I$: set of students.
- $n$: number of teams
- $J=\{1,2,\cdots,n\}$ : set of teams.
- $K$: set of characteristics.
- $a_{ik}$: student $i$'s value for characteristics $k$.
- $w_k$: the weight for characteristics $k$ in the objective.
- $L_k$: the ideal lower bound for the sum of characteristic $k$ for any team. 
- $U_k$: the ideal upper bound for the sum of characteristics $k$ for any team.

You should assume that the data is given in a excel file with the same format as the `PS10-Team-input-1.xlsx` and `PS10-Team-input-2.xlsx` files attached to this assignment. 

The sheet named "Students" encodes $I$, $K$ and $a_{ik}$'s. In the below screenshot of `PS10-Team-input-1.xlsx`, $I=\{A,B,C,D,E,F\}$, and $K=\{Person, Male, Programmer, Math, Speaking\}$.

![](PS10-Team1.png)


The sheet named "Parameters" encodes the $w_k$, $L_k$ and $U_k$ for each characteristic $k$.

![](PS10-Team2.png)

**Decision variables:**

- $x_{ij}$ : whether to assign student $i$ to team $j$. (Binary)
- $s_k$ : maximum deviation below the ideal lower bound $L_k$ for characteristic $k$. (Continuous)
- $t_k$ : maximum deviation above the ideal upper bound $U_k$ for characteristic $k$. (Continuous)

**Objective and constraints:**

$$\begin{aligned}
\text{Minimize:} && \sum_{k \in K} w_k(s_k+t_k) \\
\text{subject to:} && \\
\text{(Every person assigned)} && \sum_{j \in J} x_{ij} & = 1 && \text{For each person $i \in I$.}\\
\text{(Team balance)} && L_k - s_k \le \sum_{i \in I} a_{ik}x_{ij} & \le U_k + t_k && \text{For each team $j \in J$ and each $k \in K$.} \\
\text{(Non-negativity)} && s_k, t_k & \ge 0 && \text{for all $k$.}
\end{aligned}$$

**Write a function called "assignTeams" with the following input arguments:**

- **inputFile:** path to the input spreadsheet.
- **n:** the number of teams to divide students into.

**The function should return two variables:**

- **df:** a DataFrame with one column called "Team". The index should be the name of each individual, and the column "Team" should specify the number $j$ to which the person is assigned.
- **objval:** the optimal objective value.

For the test runs, you should download the input files attached to this exercise into the same directory as the Jupyter notebook.

In [46]:
# Write your final code here
import pandas as pd
import numpy as np
from gurobipy import Model, GRB

def assignTeams(inputFile,n):
    students = pd.read_excel(inputFile, sheet_name = "Students",index_col = 0)
    parameters = pd.read_excel(inputFile, sheet_name = "Parameters", index_col = 0)
    I = students.index
    J = range(1,n+1)
    K = students.columns
    mod = Model()
    x = mod.addVars(I,J,vtype = GRB.BINARY,name = 'x')
    s = mod.addVars(K,vtype = GRB.CONTINUOUS,name = 's')
    t = mod.addVars(K,vtype = GRB.CONTINUOUS,name = 't')
    mod.setObjective(sum(parameters.loc['Weights',k]*(s[k]+t[k]) for k in K), sense = GRB.MINIMIZE)    # shift requirement
    # Every person assigned
    for i in I:
        mod.addConstr(sum(x[i,j] for j in J) == 1,name='Every Person')
    # Team Balance
    for j in J:
        for k in K:
            mod.addConstr(sum(students.loc[i,k]*x[i,j] for i in I) >= parameters.loc['L',k] - s[k],name='Team Balance 1')
            mod.addConstr(sum(students.loc[i,k]*x[i,j] for i in I) <= parameters.loc['U',k] + t[k],name='Team Balance 2')
    mod.setParam("OutputFlag", False)
    mod.setParam('MIPGap',1e-6)
    mod.optimize()
    df = pd.DataFrame({'Names':I,'Team':np.nan})
    df.set_index('Names',inplace = True)
    for i in I:
        for j in J:
            if x[i,j].x==1:
                df.loc[i,'Team'] = j
    df['Team'] = df['Team'].astype('int')
    return(df,mod.objVal)

In [47]:
# Test run 1
# It is okay if your team numbers are different from what's below, as there are multiple optimal solutions
df,objval=assignTeams('PS10-Team-input-1.xlsx',2)
print('Optimal objective value:',objval)
df

Optimal objective value: 0.0


,Team
Names,
A,2
B,2
C,2
D,1
E,1
F,1


In [48]:
# Test run 2
# It is okay if your team numbers are different from what's below, as there are multiple optimal solutions
df,objval=assignTeams('PS10-Team-input-2.xlsx',10)
print('Optimal objective value:',objval)
df.sort_values(by='Team')

Optimal objective value: 8.0


,Team
Names,
Wayne,1
Xenna,1
Yingying,1
Ran,1
Patty,1
Catherine,2
Ouyang,2
Kathryn,2
XingZhou,2
